## Step.1 请先准备好Devign的function.json数据集

- 可从2080服务器上或https://sites.google.com/view/devign获取

## Step.2 将Devign数据集中的函数分别写入C文件中，一个函数一个.c

In [ ]:
# 查看数据情况
import json

file_path = '/mnt/sda/.../Devign/function.json'  # 修改，自己的路径

with open(file_path, 'r') as json_file:
    data = json.load(json_file)
len(data)

In [ ]:

import os
from tqdm import tqdm

vul = '/mnt/sda/.../Devign/vulc'  # 自定义缺陷保存路径
novul = '/mnt/sda/.../Devign/novulc'  # 自定义无缺陷保存路径
txt_paths = '/mnt/sda/.../Devign/devign_paths.txt'  # 自定义保存每个c文件的txt文件的路径


if not os.path.exists(vul):
    os.makedirs(vul)
if not os.path.exists(novul):
    os.makedirs(novul)


path_list = []
count = 0
for item in tqdm(data, desc="Writing to C files"):
    label = item['target']  # 数据集中标签
    fun = item['func']  # 数据集中函数的代码

    if label == 0:
        save_path = vul + str(count) + '.c'
        with open(save_path, 'w') as file:
            file.write(fun)
        path_list.append(save_path)
    else:
        save_path = novul + str(count) + '.c'
        with open(save_path, 'w') as file:
            file.write(fun)
        path_list.append(save_path)
    count += 1

with open(txt_paths, 'a') as txt_file:
    for item in path_list:
        txt_file.write(item + '\n')


## Step.3 代码正则化（可选）

注意：下面三种格式清理可自行选用，但将直接覆盖原文件，如有必要，请提前备份！！！
- delete_committe.py: 清理注释。只需要替换main()中的路径即可，该路径下应该保存着每一个.dot。
- delete_empty_line.py: 清理空行（一般情况下空行不会对joern转pdg的过程产生影响）。用法同上。
- delete_dot_patter.py: 去除类html的标签、节点label中无关的类型内容、所在行的标签等。可自定义的启发式规则。基础用法同上。

## Step.4 使用Joern生成pdg图

- Joern安装：https://docs.joern.io/installation/

#### 在使用脚本前，请检查目录结构是否符合
....../
|--Devign
|   |--devign_paths.txt
|   └── generate_dot.sh
|   └── vulc
|   └── novulc
|   └── vulcdot  # 请现在建好，脚本运行需要
|   └── novulcdot  # 请现在建好，脚本运行需要

- 1.修改.sh中txt文件的路径为上述使用的txt路径。
- 2.在命令行中运行nohup /mnt/sda/generate_dot.sh &
- 3.如果是基于Devign数据集，耐心等2-3天处理完毕。在此过程中，可进入/mnt/sda/.../joern/joern-cli目录下，使用tail -n 10 nohup.out命令查看nohup.out日志文件的后10行，若处理完成，会显示100%。

## Step.5 将1-pdg.dot提取出来

- 首先查看joern生成pdg图的过程中有没有缺失
- pdg图目录的名称是c文件名称

In [ ]:
import os
import shutil

In [ ]:
# 查看生成过程中文件是否有损失
novulcode_path = '/mnt/sda/.../Devign/novulc'
vulcode_path = '/mnt/sda/.../Devign/vulc'
novulpdgdir_path = '/mnt/sda/.../Devign/novulcdot'
vulpdgdir_path = '/mnt/sda/.../Devign/vulcdot'
if len(os.listdir(novulcode_path)) == len(os.listdir(novulpdgdir_path)):
    print(f"无缺陷代码文件数：{len(os.listdir(novulcode_path))}, 生成pdg过程中，无缺陷文件没有损失！")
else:
    print(f"无缺陷代码文件数：{len(os.listdir(novulcode_path))}, 生成pdg过程中，无缺陷文件有损失！")
if len(os.listdir(vulcode_path)) == len(os.listdir(vulpdgdir_path)):
    print(f"缺陷代码文件数：{len(os.listdir(vulcode_path))}, 生成pdg过程中，缺陷文件没有损失！")
else:
    print(f"缺陷代码文件数：{len(os.listdir(vulcode_path))}, 生成pdg过程中，缺陷文件有损失！")


- 把每个图包中的1-pdg.dot提取出来

In [ ]:

novulsingle = "/mnt/sda/.../Devign/novuldotsingle"
vulsingle = "/mnt/sda/.../Devign/vuldotsingle"

if not os.path.exists(novulsingle):
    os.makedirs(novulsingle)
if not os.path.exists(vulsingle):
    os.makedirs(vulsingle)


src_path = novulpdgdir_path  # 分别运行一下vulpdgdir_path or novulpdgdir_path
dot_dir_list = os.listdir(src_path)
for dot_dir in dot_dir_list:
    dot_dir_path = os.path.join(src_path, dot_dir)
    
    dot_path = os.path.join(dot_dir_path, "1-cpg.dot")
    if not os.path.exists(dot_path):
        continue

    des_path = os.path.join(novulsingle, dot_dir+"-1-cpg.dot")  # 分别对于修改成vulsingle or novulsingle

    shutil.copy(dot_path, des_path)

if len(dot_dir_list) == len(os.listdir(novulsingle)):  # 分别对于修改成vulsingle or novulsingle
    print(f"dot数量：{len(dot_dir_list)}, 提取1-pdg.dot完成！")
else:
    print(f"dot数量：{len(dot_dir_list)}, 提取1-pdg.dot有损失！")


- 然后就可以去生成序列了 Finish！！！